<a href="https://colab.research.google.com/github/carlmaps/Visualizaion-Projects/blob/master/Fires_from_Space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Fires from Space
This is a visualization project for the 2019-2020 Australia Bush Fire<br>
Datasource: https://www.kaggle.com/carlosparadis/fires-from-space-australia-and-new-zeland

Dataset is from NASA's observing satellite which is capable of providing real time data. NASA's Fire Information for Resource Management System (FIRMS) provides active fire data from its Moderate Resolution Imaging Spectroradiometer (MODIS) and the Visible Infrared Imaging Radiometer Suite (VIIRS).

##### Import the libraries

In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly as py
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

%matplotlib inline
init_notebook_mode(connected=True)

#set default renderer to colab
pio.renderers.default = 'colab'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##### Below are the steps to download the dataset from kaggle to google colab

In [0]:
!pip install kaggle
# Import kaggle.json from google drive
# This snippet will output a link which needs authentication from any google account
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
    q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
# print(kaggle_api_key)
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

! mkdir ~/.kaggle
! cp /content/.kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d carlosparadis/fires-from-space-australia-and-new-zeland

!mkdir bushfire
!unzip fires-from-space-australia-and-new-zeland.zip -d bushfire

In [0]:
# Import kaggle.json from google drive
# This snippet will output a link which needs authentication from any google account
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
    q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
# print(kaggle_api_key)
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

Download 100%.


##### Load the dataset to a pandas dataframe

In [0]:
df1 = pd.read_csv('/content/bushfire/fire_archive_V1_96617.csv')
df2 = pd.read_csv('/content/bushfire/fire_nrt_V1_96617.csv')

In [0]:
df1.columns

Index(['latitude', 'longitude', 'bright_ti4', 'scan', 'track', 'acq_date',
       'acq_time', 'satellite', 'instrument', 'confidence', 'version',
       'bright_ti5', 'frp', 'type'],
      dtype='object')

In [0]:
df2.columns

Index(['latitude', 'longitude', 'bright_ti4', 'scan', 'track', 'acq_date',
       'acq_time', 'satellite', 'instrument', 'confidence', 'version',
       'bright_ti5', 'frp', 'daynight'],
      dtype='object')

Description of the columns


| Attribute | Short Description | Long Description  |
|---|---|---|
| Latitude|Latitude  |Center of nominal 375 m fire pixel   |
| Longitude  |Longitude   |Center of nominal 375 m fire pixel   |
| Bright_TI4  |Brightness temperature I-4   |VIIRS I-4 channel brightness temperature of the fire pixel measured in Kelvin.   |
| Scan  |Along Scan pixel size   |The algorithm produces approximately 375 m pixels at nadir. Scan and track reflect actual pixel size.   |
| Track  |Along Track pixel size   |The algorithm produces approximately 375 m pixels at nadir. Scan and track reflect actual pixel size.   |
| Acq_Date  |Acquisition Date   |Date of VIIRS acquisition.   |
| Acq_Time  |Acquisition Time   |Time of acquisition/overpass of the satellite (in UTC).   |
| Satellite  |Satellite   |N= Suomi National Polar-orbiting Partnership (Suomi NPP), 1=NOAA-20 (designated JPSS-1 prior to launch)   |
| Instrument  |   |   |
| Confidence  |Confidence   |This value is based on a collection of intermediate algorithm quantities used in the detection process. It is intended to help users gauge the quality of individual hotspot/fire pixels. Confidence values are set to low, nominal and high. Low confidence daytime fire pixels are typically associated with areas of sun glint and lower relative temperature anomaly (<15K) in the mid-infrared channel I4. Nominal confidence pixels are those free of potential sun glint contamination during the day and marked by strong (>15K) temperature anomaly in either day or nighttime data. High confidence fire pixels are associated with day or nighttime saturated pixels.   |
| Version  |Version (Collection and source)   |Version identifies the collection (e.g. VIIRS Collection 1) and source of data processing: Near Real-Time (NRT suffix added to collection) or Standard Processing (collection only). <br>
"1.0NRT" - Collection 1 NRT processing.
"1.0" - Collection 1 Standard processing   |
| Bright_TI5  |Brightness temperature I-5   |I-5 Channel brightness temperature of the fire pixel measured in Kelvin.   |
| FRP  |Fire Radiative Power   |FRP depicts the pixel-integrated fire radiative power in MW (megawatts).   |
| Daynight  |Day or Night   |D= Daytime fire, N= Nighttime fire   |

In [0]:
df_VIIRS = pd.concat([df1, df2], sort=True)

In [0]:
df_VIIRS.shape

(1141035, 15)

In [0]:
df_VIIRS.columns

Index(['acq_date', 'acq_time', 'bright_ti4', 'bright_ti5', 'confidence',
       'daynight', 'frp', 'instrument', 'latitude', 'longitude', 'satellite',
       'scan', 'track', 'type', 'version'],
      dtype='object')

In [0]:
df_VIIRS = df_VIIRS[['latitude','longitude','acq_date','frp']]
df_VIIRS.head()

,latitude,longitude,acq_date,frp
0,-34.45902,150.88040,2019-08-01,5.8
1,-31.70724,151.07191,2019-08-01,10.4
2,-31.39626,149.65253,2019-08-01,5.6
3,-30.39774,152.06432,2019-08-01,9.1
4,-31.20248,151.89766,2019-08-01,9.4


In [0]:
#sort the data by acq_date
df_VIIRS.sort_values(by='acq_date', ascending=True)

,latitude,longitude,acq_date,frp
0,-34.45902,150.88040,2019-08-01,5.8
2482,-14.16059,130.05383,2019-08-01,1.1
2483,-14.16737,130.05188,2019-08-01,1.2
2484,-14.16672,130.01665,2019-08-01,3.4
2485,-14.15595,130.03313,2019-08-01,1.2
...,...,...,...,...
952497,-16.05202,144.75102,2020-01-11,6.4
952496,-16.05298,144.74356,2020-01-11,8.9
952495,-16.04700,144.75786,2020-01-11,9.8
953440,-17.85031,124.07442,2020-01-11,5.4


In [0]:
df_VIIRS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1141035 entries, 0 to 956256
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   latitude   1141035 non-null  float64
 1   longitude  1141035 non-null  float64
 2   acq_date   1141035 non-null  object 
 3   frp        1141035 non-null  float64
dtypes: float64(3), object(1)
memory usage: 43.5+ MB


In [0]:
df_VIIRS.describe()

,latitude,longitude,frp
count,1.141035e+06,1.141035e+06,1.141035e+06
mean,-2.620565e+01,1.417957e+02,1.443618e+01
std,8.416856e+00,1.080249e+01,4.229887e+01
min,-4.321232e+01,1.134470e+02,0.000000e+00
25%,-3.278702e+01,1.318983e+02,2.000000e+00
50%,-2.943247e+01,1.473232e+02,5.100000e+00
75%,-1.665093e+01,1.506748e+02,1.230000e+01
max,-9.394060e+00,1.535211e+02,1.071720e+04


In [0]:
df_VIIRS.frp.max()

10717.2

In [0]:
df_VIIRS[df_VIIRS['frp'] == 10717.2]

,latitude,longitude,acq_date,frp
942932,-36.97858,146.71938,2020-01-08,10717.2


In [0]:
lat_max = df_VIIRS.loc[df_VIIRS['frp'].idxmax()]['latitude']

latitude       -36.9786
longitude       146.719
acq_date     2020-01-08
frp             10717.2
Name: 942932, dtype: object

In [0]:
lat_max = df_VIIRS.loc[df_VIIRS['frp'].idxmax()]['latitude']
long_max = df_VIIRS.loc[df_VIIRS['frp'].idxmax()]['longitude']

#### Using Folium for map visualization
Import the folium package


In [0]:
import folium
from folium import plugins

Generate Map of Autralia. Location is -27,132

In [0]:
m = folium.Map(location=[-27, 132], zoom_start=4.4, tiles='Stamen Terrain', control_scale=True)
m

Create marker for the highest frp

In [0]:
folium.Marker(location=[lat_max, long_max], tooltip= 'frp: ' + str(df_VIIRS.loc[df_VIIRS['frp'].idxmax()]['frp'])
+ '<br/> date: ' + str(df_VIIRS.loc[df_VIIRS['frp'].idxmax()]['acq_date']), icon=folium.Icon(color='red', icon='fire'),).add_to(m)

m

Plotting the top 10 highest frp

In [0]:
#getting the top highest

df_top10 = df_VIIRS.sort_values(by='frp', ascending=False).head(10)

for i in range(len(df_top10)):
  folium.Marker(location=[df_top10.iloc[i]['latitude'], df_top10.iloc[i]['longitude']], tooltip= 'frp: ' + str(df_VIIRS.iloc[i]['frp'])
+ '<br/> date: ' + str(df_VIIRS.iloc[i]['acq_date']), icon=folium.Icon(color='red', icon='fire'),).add_to(m)

m

In [0]:
from folium.plugins import HeatMapWithTime

In [0]:
df_VIIRS.groupby(['latitude', 'longitude']).sum().reset_index()

,latitude,longitude,frp
0,-43.21232,146.98082,8.3
1,-43.17590,147.03905,0.5
2,-43.14765,147.76562,1.0
3,-43.14532,147.76862,1.4
4,-43.14526,147.76770,1.2
...,...,...,...
1141024,-10.07851,142.17274,2.4
1141025,-9.40703,142.66313,7.7
1141026,-9.39655,142.64885,28.3
1141027,-9.39650,142.64742,15.4


In [0]:
#df_VIIRS.acq_date.sort_values().unique()

df_day_list = []
for day in df_VIIRS.acq_date.sort_values().unique():
        df_day_list.append(df_VIIRS.loc[df_VIIRS.acq_date == day, ['acq_date','latitude', 'longitude', 'frp']].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist())

#df_day_list

In [0]:
m = folium.Map(location=[-27, 132], zoom_start=4.4, tiles='Stamen Terrain', control_scale=True)
HeatMapWithTime(data=df_day_list, index=list(df_VIIRS.acq_date.sort_values().unique()), auto_play=True,radius=3.5, gradient={0.2: 'blue', 0.4: 'lime', 0.6: 'orange', 1: 'red'}, min_opacity=0.2, max_opacity=0.8, use_local_extrema=True).add_to(m)

m.save (r'Time.html')